# Analyzing an MD Simulation from ChemCompute Using MD Analysis
by Gianmarc Grazioli, Ph.D.  

### Overview
This project gives you the opportunity to learn how to: 
1) Run molecular dynamics simulations using the NAMD software package through you browser using ChemCompute.
2) Do some analysis of the simulation data using the MD Analysis Python library

In this Jupyter notebook, you will use the MDAnalysis Python library to analyze data from a short molecular dynamics simulation of your assigned protein solvated in water. 

### YouTube Video
This activity has an accompanying YouTube video here: https://youtu.be/jrv_cAG514k 

### Installing Libraries
If you have never used the Python libraries MDAnalysis or nglview, first uncomment and then execute the cells beginning with __pip install__ below to install these libraries onto your computer. Once installed, you can comment these cells back out.

In [1]:
#pip install --user MDAnalysis MDAnalysisTests  #analyze molecular dynamics simulation#

In [2]:
#pip install nglview   #visualizing md analysis like vmd#

In [3]:
#pip install ipywidgets

### Import the necessary libraries:

In [11]:
import pickle
import MDAnalysis as mda
import numpy as np
import matplotlib.pyplot as plt
import nglview as nv
import os
import pandas as pd

### Load and visualize your assigned protein:
1) Download your protein's pdb file from the Protein Data Bank (PDB) https://www.rcsb.org/
2) Drag your pdb file from your downloads into the same directory where this Jupyter notebook is located (do not leave this Jupyter notebook in your Downloads directory).
3) Modify the code below so that your pdb file name replaces your.pdb.

Note the line: u1 = mda.Universe("your.pdb")
This line creates an instanced named u1 of an object native to MDAnalysis called a Universe. The Universe is a convenient object that can hold a variety of important attributes for MD simulations (number of atoms, all the atom names, positions, etc.) and possesses a variety of methods, such as the ability to add angles of interest to be calculated.

4) Excecute the cell below to load the pdb structure into this notebook and view it.
5) Try moving the structure around to get a better feel for it spatially.

In [32]:
# Load the PDB structure
u1 = mda.Universe("Chloroform.pdb")
#Universe is an object that recognize the file type that describe the molecule

# Create a view using nglview
view = nv.show_mdanalysis(u1)

# Display the visualization in the notebook
view

<PDBReader Chloroform.pdb with 1 frames of 5 atoms>


### Measure the size of your assigned protein:
1) Access the positions of all the atoms using the u1 object and then get the position of the atom with the lowest x,y, and z coordinate with the command: min_coords = u1.atoms.positions.min(axis=0)
2) Create a variable max_coords, and modify the command above to set it equal to the coordinates of the atom with the highest x,y, and z coordinates.
3) Take the difference between these two vectors to determine the dimensions of the structure.

In [27]:
# Calculate the bounding box of the structure
min_coords = u1.atoms.positions.min(axis=0)
max_coords = u1.atoms.positions.max(axis=0)

max_coords - min_coords

array([2.8439999, 2.58     , 2.071    ], dtype=float32)

### Running a Molecular Dynamics simulation on ChemCompute:
1) Sign up for a free acount: https://chemcompute.org/, then login to your account.
2) Go to the NAMD tab on their website.
3) Click submitting a job.
4) Enter your assigned Protein Data Bank (PDB) ID (e.g. 1YRF)
5) Choose the size of your simulated water box, ensuring that all side lengths are at least double the length of the longest dimension of the protein.
6) Run the simulation with the rest of the parameters set to their default values.

You should see spinning gears to indicate that your simulation has started running. It is a good idea to copy your job ID number, in case you need to look up your results later. Be forewarned, especially if your protein structure is large, __this could take a few hours!__ 

7) Once the gears stop spinning, click Download Output File, and move the downloaded zip file into the same directory as this Jupyter notebook. Then unzip the file and rename the directory simData.

### Loading Simulation Data for Ubiquitin in Explicit Water
Here we will again take advantage of the extremely helpful MDAnalysis library for loading all of the simulation data into a single object called a __Universe__.  

5) Define a path to the trajectory data below using os.path.join, then load the files box_wb.psf and md.dcd into an MDAnalysis __Universe__ called u.

In [7]:
PSF = os.path.join()    #protein structure file
DCD = os.path.join()    #dynamic coordinate data (moving coordinates)
u = mda.Universe(PSF, DCD)

### Visualizing the Trajectory using NGLview

6) Use the nglview function __show_mdanalysis()__ to operate on the universe __u__ to create an object called wBox. Then use the __add_representation()__ function to add a representation of the water molecules to the wBox object. Use the "licorice" visualization style, your selection should of course be "water," and set the color to cyan. Once you have defined the wBox object, execute a new code cell with just wBox in it, and you should be treated to a very cool animated visualization of the simulation data!  

In [8]:
# wBox = nv.show_mdanalysis(u)
# wBox.add_representation()

In [9]:
# wBox

### Analyzing the Simulation Data using MDAnalysis

7) There is a great MDAnalysis demonstration on YouTube that I recommend you check out here:\
https://youtu.be/zVQGFysYDew \
\
In particular, there is a demonstration where they plot RMSF (root mean squared fluctuations) of the alpha carbons of a protein changing as a function of time. The RMSF plot basically shows you which alpha carbons are the most "wiggly" as you move down the protein's backbone. The code written in the YouTube video uses a trick whereby averaging is done while iterating over the loop by reweighting based on how far along in the loop you are (e.g. note that they use enumerate to get the iterator k, and then reweight so that the 2nd step is weight 1/2, then 3rd is 2/3, then 3/4, etc.). If you'd like to code it their way, simply copy their code in a cell below, just make sure you read it carefully and are able to explain to someone else how it works! The method they use is great for efficiency, and is often used when massive data sets cannot be loaded in their entirety, so one must calculate averages while loading the data piece by piece. In our case, we are working with a very small data set by molecular dynamics standards, so we can easily load the full data set and carry out averaging the standard way (do the full sum, then divide). In a cell below, write a code that uses the MDAnalysis library to create a plot of RMSF as a function of residue number, like the one in their video, from your trajectory.

If you decide to try writing your own code for calculating RMSF, here are some steps:

- Store all the alpha carbon positions in an array called ca_positions by looping over the trajectory object accessible from the Universe object (u.trajectory).
- Store the mean positions of these alpha carbons in array mean_positions by using the mean method native to arrays.
- Calculate the squared displacements by subtracting the mean positions from the ca positions and squaring that difference. __Note:__ since ca_positions and mean_positions are numpy arrays, you can do the subtraction and squaring as vector operations, so no need to write a loop of any sort.
- Calculate the mean squared displacement by taking the mean of the squared_displacements array. __Note:__ the final squared displacements involve summing over the squared displacements in x,y, and z. If that seems odd, consider that the definition of Euclidean distance is: $$r = \sqrt{\Delta x^2 + \Delta y^2 + \Delta z^2},$$ so it is really not so strange since $$r^2 = \Delta x^2 + \Delta y^2 + \Delta z^2$$.
- The only part of the calculation at this point is to take the square root, since it is a __root__ mean squared fluctuation, afterall! Thus the last step is, rmsf = np.sqrt(mean_squared_displacements).
- As far as plotting goes, use matplotlib.pyplot to plot RMSF the same way Dr. Beckstein does it in his video.

In [10]:
# Step 1: Load all positions of CA atoms over all trajectory frames
# ca = # select all the carbon atoms first using select_atoms()
# ca_positions = 

# Step 2: Compute the mean position for each atom across all frames
# mean_positions = 

# Step 3: Compute RMSF for each atom
# squared_displacements = 
# mean_squared_displacements = 
# rmsf = 

# Step 4: Plot RMSF against residue IDs
# plt.plot(ca.residues.resids, rmsf)
# plt.plot(ca.residues.resids, rmsf, 'o')
# plt.xlabel("Alpha Carbon Resid")
# plt.ylabel("RMSF")
# plt.show()

Now that we have quantified how much each amino acid's alpha carbon is jiggling about during the simulation, let us next create an animated molecular visualization that highlights the two most jiggly amino acids. To do this, let's programmatically obtain the residue IDs (resid) of the top 2 most jiggly alpha carbons:

8) The DataFrame object from the Pandas library is a fantastic structure for storing and analyzing data, so let's store the RMSF data in a DataFrame, where the first column is the resid and the second column is RMSF. 
9) Once the DataFrame is created, sort the rows in place in descending order by rmsf, and then take a look at the first few rows and record the top 2 most jiggly residues below:

__Most jiggly residue =__ \
__Second most jiggly residue =__ 

In [11]:
#resid_with_rmsf = pd.DataFrame()

9) Now that you know which two alpha carbons are jiggling the most:

> 9.a) Use nglview to create a visualization where the two most jiggly residues are highlighted by coloring them differently from the rest of the protein (leave the waters out for clarity this time).

> 9.b) Add a markdown cell below your visualization where you describe, in your own words, why you think these residues are more jiggly than the rest.

In [12]:
# jiggles = nv.show_mdanalysis(u)
# jiggles.add_representation()

In [13]:
# jiggles